In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.text import *
import pickle
import pandas as pd
import html

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH=Path('/home/paperspace/data/mimic-iii')
LM_PATH=PATH/'lm_word_level'

In [9]:
trn_ids = np.load(LM_PATH/'trn_ids_concat.npy')
val_ids = np.load(LM_PATH/'val_ids_concat.npy')

In [4]:
with open(LM_PATH/'itos.pickle', 'rb') as handle:
    itos = pickle.load(handle)
    
vocab_size=len(itos)

In [7]:
em_sz,nh,nl = 400,1150,3
bptt, bs = 70, 64

In [10]:
trn_dl = LanguageModelLoader(trn_ids, bs, bptt)
val_dl = LanguageModelLoader(val_ids, bs, bptt)
md = LanguageModelData(PATH, 0, vocab_size, trn_dl, val_dl, bs=bs, bptt=bptt)

In [11]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.5

In [12]:
opt_fn = partial(optim.Adam, betas=(0.8,0.99))
learner= md.get_model(opt_fn, em_sz, nh, nl,
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])
learner.metrics = [accuracy]
learner.clip=0.12
learner.unfreeze()
learner.reg_fn=partial(seq2seq_reg, alpha=2, beta=1)
wd = 1e-7

In [ ]:
learner.lr_find(end_lr=1000)

In [ ]:
learner.sched.plot()

In [10]:
lr = 10e-3

In [11]:
learner.fit(lr, 1, cycle_len=10, use_clr_beta=(10,10,0.95,0.85))

epoch      trn_loss   val_loss   accuracy                         
    0      2.635664   2.427942   0.540275  
    1      2.551668   2.369839   0.547285                         
    2      2.572614   2.381757   0.546985                         
    3      2.568972   2.412489   0.543766                         
    4      2.614528   2.41644    0.543442                         
    5      2.539698   2.358792   0.550942                         
    6      2.484577   2.30351    0.558644                         
    7      2.431909   2.254647   0.565642                         
    8      2.387832   2.215251   0.571395                         
    9      2.359126   2.19928    0.573917                         



[array([2.19928]), 0.5739172494452303]

In [ ]:
learner.sched.plot_loss(n_skip=20000)

In [19]:
learner.save('word_level_default_tokenization')

In [13]:
learner.load('word_level_default_tokenization')

In [25]:
m = learner.model

with open(LM_PATH/'stoi.pickle','rb') as f:
    stoi = pickle.load(f)

def sample_model(m, s, l=100):
    s_toks = Tokenizer().proc_text(s)
    s_nums = [stoi[i] for i in s_toks]
    s_var = V(np.array(s_nums))[None]

    m[0].bs=1
    m.eval()
    m.reset()

    res, *_ = m(s_var)
    print('...', end='')

    for i in range(l):
        r = torch.multinomial(res[-1].exp(), 2)
        #r = torch.topk(res[-1].exp(), 2)[1]
        if r.data[0] == 0:
            r = r[1]
        else:
            r = r[0]
        
        word = itos[to_np(r)[0]]
        res, *_ = m(r[0].unsqueeze(0))
        print(word, end=' ')
    m[0].bs=bs

In [46]:
s = 'cardiovascular system '
sample_model(m, s)

..., deviation.non-specific occulsion for cardiac resection . lastname * * ] now her prognosis is slightly difficult for d / c to rehab if patient can not recover - i c [ * * name8 ( md ) 24 * * ] team care . t_up events - dc po abx as patient on glargine drip for trach t_up icu care nutrition : tfs continues , sedation / t_up npo for procedure glycemic control : insulin sliding scale lines : multi lumen - [ * * 2156 - 12 - 28 * * ] 08:26 pm dialysis catheter - [ 

In [51]:
s = 'blood loss '
sample_model(m, s)

...of bilateral leg veins that was ruled out by inline ) it . had at thistime obtained w / o sign of carotid stenosis which represents anterior ventricular serosanguinous values , no obvious source of infection seen . t_up cxr suggestive of normal superior months.disp:*60 toxins . 2057 and active vs , pulses normal , no focal deficits . per team thought to be due to bloodculture , that is not consistent with an ischemic event and considering slow outpatient monitoring was felt to be secondary to her recent 4:42 ( right sided thigh hypotonic ) . iv access was 